In [55]:
import pandas as pd
import re, os, operator
import numpy as np

import spacy

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#### Reading and cleaning the dataframe

In [2]:
# read csv file 'description.csv
df = pd.read_csv('description_.csv', header=None)

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
df.head(2)

,0,1,2,3
0,https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/wonderland/,Med 350 millioner kroner i omsetning er Wonderland en av de største sengeprodusentene i Norge. Nå vil selskapet på Åndalsnes øke eksporten til Europa.,"En høyteknologisk robotlinje sørger for at skumgummi, fjærer og andre komponenter plasseres riktig og blir til en «liggeflate» i andre enden. For femti år siden ble den første skummadrassen laget, og nå produserer Wonderland over 90 000 madrasser i året i fabrikkanlegget på Åndalsnes.\n","\n- I robotlinja har vi mulighet til å produsere to-tre ganger flere madrasser om markedet skulle ønske det. Vi tenker at en vekst skal skje på eksport, der Norden og deler av EU vil være vår prioritet, sier administrerende direktør i Wonderland, Lars Stenerud.\nSengeprodusenten kan se tilbake på fem tiår med skiftende eierskap, krevende markeder og hard konkurranse. Wonderland har klart seg godt og vil nå vokse videre. De eksporterer noe til Norden allerede, men har et mål om å øke eksporten til Europa. Ved å delta i Innovasjon Norges program for eksportutvikling, Global Gateways, får de tett oppfølging gjennom hele eksportprosessen. Vi bidrar med kompetanse, rådgivning og nettverk for å sikre raskere tilgang til nye markeder.\n \n"
1,https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/c-food-norway/,Bedre utnyttelse av fiskefangst la grunnlaget for C Food Norway. Nå bistår Innovasjon Norge bedriften med å utvikle en bærekraftig forretningsmodell for bruk av fiskehoder.,"Norge er en stor og viktig sjømatnasjon, men vi har potensial for å bli både mer bærekraftig og skape større verdier av det vi fangster. Målet er at alt vi fangster skal utnyttes, og ikke noe av fangsten skal gå til spille.\nDrahjelp ut i verden\n","\nC Food Norway har jobbet mye med å utnytte rester av hvitfisk til mat og helsekostprodukter siden etableringen i 2013. Ålesund-bedriften har kommet langt med å utnytte fiskerygger og svømmeblærer, og er i gang med å utvikle bærekraftige og lønnsomme verdikjeder for fiskehoder. I mange land blir nettopp hodet betraktet som den beste delen av fisken.\n- For å lykkes med nye sjømatprodukter i et globalt marked, trenger vi å lære andre lands kulturer og matvaner. Innovasjon Norges utekontor er viktige samarbeidspartnere for å øke bedriftenes markedskunnskap og kulturforståelse. Utekontorene gir nødvendig drahjelp for å introdusere produktene i mulige markeder, forteller Claus Opshaug, daglig leder i C Food Norway.\nStort potensial i Asia\nI prosjektet kalt «Fishhead» skal bedriften utvikle et nytt grønt forretningskonsept i tett og langsiktig samarbeid med leverandører, utstyrsprodusenter og kunder. Planen er å produsere både tørkede, fryste, salta og hermetiserte hvitfiskhoder til bruk i måltider.\n- Dette er et interessant og et viktig prosjekt med stort potensial, spesielt i Asia. C Food Norway er en foregangsbedrift for bransjen, sier Kari-Anne Lade Gjørvad, kunderådgiver for Innovasjon Norge Møre og Romsdal.\n \n"


In [5]:
df.shape

(149, 4)

In [6]:
# create a new column with last word between '/' from each link
# this is the product name
df[4] = df[0].str.split('/').str[-2]

In [7]:
# create list from column with product names
product_names = df[4].tolist()

In [8]:
len(product_names)

149

In [9]:
for product in product_names:
    print(product)

wonderland
c-food-norway
exotek
miljo-norge
oceangrid
appsens
beefutures-holding
oso-hotwater
gronn-gjodsel
gudbrandsdalens-uldvarefabrik
diffia
synergy-sky
windsim
trysil-maskin
slettvoll-mobler
novelda
hardanger-siderprodusentlag
ducky
pixii
nammo-raufoss
tunable
sunlit-sea
ca-technology-systems
origo-solutions
vil-gjore-kysten-utslippsfri-med-hydrogen
easee
kongsberg-satellite-services
zooca
arbaflame
bright-products
yedlik
falstadsenteret-drift
elliptic-labs
drytech
innovern
deep-purple
hycast
link-analytix
touchnetix
greentech-innovators
cycled-technologies
innsep
fixrate
secure-practice
corvus-energy-og-lmg-marin
oceanize
checkware
jentene-pa-tunet
turbulent-flux
leroy-sjotroll
waterise
exentric
e-plug
vestre
beyonder
planktonic
cinderella-eco-group
borregaard
sevendof
kobler
bliksund
csub
onio
gardsbrenneriet
airthings
skala-maskon
decon-x-international
nordkontakt
anue
bridgehill
weisstech
elop
laviair
vixel
attensi
oliasoft
marine-recycling-cluster
spilltech
maritech-systems
w

In [10]:
# copy dataframe to continue with concatenation and cleaning
df_copy = df.copy()

In [11]:
# concatenate columns 1, 2 and 3 into a new column
df_copy[5] = df_copy[1] + ' ' + df_copy[2] + ' ' + df_copy[3]

In [12]:
# delete columns 1, 2 and 3
df_copy.drop(columns=[1, 2, 3], inplace=True)

In [13]:
# replace NaN values with empty string in all columns
df_copy.fillna('', inplace=True)

In [14]:
# function to clean the description text
# creates a new column with cleaned text
def cleanhtml():
    # remove html tags
    cleanr = re.compile('<.*?>')
    
    rows = df_copy[5]
    listcleantext = []
    for r in rows:
        cleantext = re.sub(cleanr, '', r)
        
        # replace new line \n with empty space
        cleantext = cleantext.replace('\n', ' ')
        
        # replace &quot; with single quote
        cleantext = cleantext.replace('&quot;', "''")
        
        # replace &nbsp with empty space
        cleantext = cleantext.replace('&nbsp', ' ')
        
        listcleantext.append(cleantext)
    df_copy[6] = listcleantext
    return

In [15]:
cleanhtml()

In [17]:
# remove column 5
df_copy.drop(columns=[5], inplace=True)

In [18]:
df_copy.head()

0  \
0     https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/wonderland/   
1  https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/c-food-norway/   
2         https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/exotek/   
3    https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/miljo-norge/   
4      https://www.innovasjonnorge.no/no/tjenester/kundehistorier/2022/oceangrid/   

               4  \
0     wonderland   
1  c-food-norway   
2         exotek   
3    miljo-norge   
4      oceangrid   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

#### Creating a bag of word from all descriptions

We group all our descriptions in a list of lists.

In [22]:
descriptions_list = df_copy[6].values.tolist()

In [23]:
# quick example with first list element
print(descriptions_list[0:1])

['Med 350 millioner kroner i omsetning er Wonderland en av de største sengeprodusentene i Norge. Nå vil selskapet på Åndalsnes øke eksporten til Europa. En høyteknologisk robotlinje sørger for at skumgummi, fjærer og andre komponenter plasseres riktig og blir til en «liggeflate» i andre enden. For femti år siden ble den første skummadrassen laget, og nå produserer Wonderland over 90\xa0000 madrasser i året i fabrikkanlegget på Åndalsnes.   - I robotlinja har vi mulighet til å produsere to-tre ganger flere madrasser om markedet skulle ønske det. Vi tenker at en vekst skal skje på eksport, der Norden og deler av EU vil være vår prioritet, sier administrerende direktør i Wonderland, Lars Stenerud. Sengeprodusenten kan se tilbake på fem tiår med skiftende eierskap, krevende markeder og hard konkurranse. Wonderland har klart seg godt og vil nå vokse videre. De eksporterer noe til Norden allerede, men har et mål om å øke eksporten til Europa. Ved å delta i Innovasjon Norges program for ekspo

We use regex (re) to do some more cleaning

In [24]:
descriptions_list_re = []

for sentence in range(0, len(descriptions_list)):
    
    # Remove single characters from the start
    newcol = re.sub(r'\^[a-zA-Z]\s+', ' ', str(descriptions_list[sentence]))
    
    # Remove all the special characters
    newcol = re.sub(r'\W', ' ', newcol)

    # Substituting multiple spaces with single space
    newcol = re.sub(r'\s+', ' ', newcol, flags=re.I)

    descriptions_list_re.append(newcol)

In [25]:
print(descriptions_list_re[0:1])

['Med 350 millioner kroner i omsetning er Wonderland en av de største sengeprodusentene i Norge Nå vil selskapet på Åndalsnes øke eksporten til Europa En høyteknologisk robotlinje sørger for at skumgummi fjærer og andre komponenter plasseres riktig og blir til en liggeflate i andre enden For femti år siden ble den første skummadrassen laget og nå produserer Wonderland over 90 000 madrasser i året i fabrikkanlegget på Åndalsnes I robotlinja har vi mulighet til å produsere to tre ganger flere madrasser om markedet skulle ønske det Vi tenker at en vekst skal skje på eksport der Norden og deler av EU vil være vår prioritet sier administrerende direktør i Wonderland Lars Stenerud Sengeprodusenten kan se tilbake på fem tiår med skiftende eierskap krevende markeder og hard konkurranse Wonderland har klart seg godt og vil nå vokse videre De eksporterer noe til Norden allerede men har et mål om å øke eksporten til Europa Ved å delta i Innovasjon Norges program for eksportutvikling Global Gatewa

Creating bag of words from list of lists

In [37]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=False keeps accents on Norwegian letters

bag_of_words = list(sent_to_words(descriptions_list_re))

In [38]:
print(bag_of_words[0:1])

[['med', 'millioner', 'kroner', 'omsetning', 'er', 'wonderland', 'en', 'av', 'de', 'største', 'norge', 'nå', 'vil', 'selskapet', 'på', 'åndalsnes', 'øke', 'eksporten', 'til', 'europa', 'en', 'høyteknologisk', 'robotlinje', 'sørger', 'for', 'at', 'skumgummi', 'fjærer', 'og', 'andre', 'komponenter', 'plasseres', 'riktig', 'og', 'blir', 'til', 'en', 'liggeflate', 'andre', 'enden', 'for', 'femti', 'år', 'siden', 'ble', 'den', 'første', 'skummadrassen', 'laget', 'og', 'nå', 'produserer', 'wonderland', 'over', 'madrasser', 'året', 'fabrikkanlegget', 'på', 'åndalsnes', 'robotlinja', 'har', 'vi', 'mulighet', 'til', 'produsere', 'to', 'tre', 'ganger', 'flere', 'madrasser', 'om', 'markedet', 'skulle', 'ønske', 'det', 'vi', 'tenker', 'at', 'en', 'vekst', 'skal', 'skje', 'på', 'eksport', 'der', 'norden', 'og', 'deler', 'av', 'eu', 'vil', 'være', 'vår', 'prioritet', 'sier', 'administrerende', 'direktør', 'wonderland', 'lars', 'stenerud', 'kan', 'se', 'tilbake', 'på', 'fem', 'tiår', 'med', 'skiftend

Removing stop words from bag of words

In [41]:
stop_words = stopwords.words('norwegian')
# stop_words.extend(['på'])
# --> we can add extra words here that we want to remove from the bag of words

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

bag_of_words_no_stop = remove_stopwords(bag_of_words)

In [42]:
print(bag_of_words_no_stop[0:1])

[['millioner', 'kroner', 'omsetning', 'wonderland', 'største', 'norge', 'selskapet', 'åndalsnes', 'øke', 'eksporten', 'europa', 'høyteknologisk', 'robotlinje', 'sørger', 'skumgummi', 'fjærer', 'andre', 'komponenter', 'plasseres', 'riktig', 'liggeflate', 'andre', 'enden', 'femti', 'år', 'første', 'skummadrassen', 'laget', 'produserer', 'wonderland', 'madrasser', 'året', 'fabrikkanlegget', 'åndalsnes', 'robotlinja', 'mulighet', 'produsere', 'to', 'tre', 'ganger', 'flere', 'madrasser', 'markedet', 'ønske', 'tenker', 'vekst', 'skje', 'eksport', 'norden', 'deler', 'eu', 'prioritet', 'sier', 'administrerende', 'direktør', 'wonderland', 'lars', 'stenerud', 'se', 'tilbake', 'fem', 'tiår', 'skiftende', 'eierskap', 'krevende', 'markeder', 'hard', 'konkurranse', 'wonderland', 'klart', 'godt', 'vokse', 'videre', 'eksporterer', 'norden', 'allerede', 'mål', 'øke', 'eksporten', 'europa', 'delta', 'innovasjon', 'norges', 'program', 'global', 'gateways', 'får', 'tett', 'oppfølging', 'gjennom', 'hele', 

In [43]:
len(bag_of_words_no_stop) # --> this should still return the same length of our original dataframe :)

149

##### Processing Bag of Words using spaCy library

In [49]:
# path to our spaCy language model
path_spacy_model = '/Users/Anne/anaconda3/envs/python37/lib/python3.7/site-packages/nb_core_news_md/nb_core_news_md-3.0.0'

# loading the model
nlp = spacy.load(path_spacy_model, disable=['parser', 'ner'])

In [50]:
type(nlp)

spacy.lang.nb.Norwegian

Reducing the term count

We have already removed our stop words.<br>
The lemmatization will allow us to select a specific type of word, for instance 'NOUN'

In [51]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))    # remember: nlp is defined in our previous cell, it is the spaCey language model
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [52]:
bag_of_words_no_stop_lemma = lemmatization(bag_of_words_no_stop, allowed_postags=['NOUN'])

In [53]:
print(bag_of_words_no_stop_lemma[0:1])

[['million', 'krone', 'omsetning', 'wonderland', 'norge', 'selskap', 'åndalsnes', 'eksport', 'europe', 'robotlinje', 'skumgummi', 'fjære', 'komponent', 'ende', 'år', 'skummadrasse', 'lag', 'wonderland', 'madrass', 'år', 'fabrikkanlegg', 'åndalsnes', 'mulighet', 'ganger', 'madrass', 'marked', 'ønske', 'vekst', 'eksport', 'norde', 'prioritet', 'direktør', 'tiår', 'eierskap', 'marked', 'konkurranse', 'wonderland', 'norde', 'mål', 'eksport', 'europe', 'innovasjon', 'program', 'gateways', 'oppfølging', 'kompetanse', 'rådgivning', 'nettverk', 'tilgang', 'marked']]


Inspect most frequest terms from our lemmatized bag of words

In [66]:
def detoken(texts):
    """detokenizer"""
    texts_out = []
    for sent in texts:
        texts_out.append(" ".join(s for s in sent))
    return texts_out

bag_of_words_no_stop_lemma_test = detoken(bag_of_words_no_stop_lemma)    # reassembling lists of tokens into continuous lists of strings

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(bag_of_words_no_stop_lemma_test)
freq = np.ravel(X.sum(axis=0)) 

vocab = [v[0] for v in sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))]
fdist = dict(zip(vocab, freq))

# Save all terms as CSV file to inspect all

with open('term_dict.csv', 'w', encoding='utf-8') as f:
    for key in fdist.keys():
        f.write("%s,%s\n"%(key,fdist[key]))

In [69]:
# Reading term_dict csv file into a pandas and sort the df by frequencies 
# to see which words are very frequent (ubiquitous).
df_term_dict = pd.read_csv('term_dict.csv', names=['token', 'frequency'])
df_term_dict.sort_values('frequency', ascending= False, inplace=True)

In [70]:
df_term_dict.head(20)

,token,frequency
1505,innovasjon,730
2358,norge,563
2985,selskap,385
2059,marked,291
2029,løsning,269
256,bedrift,246
2697,prosjekt,190
3446,teknologi,189
2672,produkt,177
3328,støtte,176
